# Customer Shopping Behavior Analysis — SQL Insight Report

## Goal of This Notebook  
This notebook uses **MySQL + Python (SQLAlchemy)** to:

1. Compute essential customer KPIs.  
2. Analyze spending patterns by category, season, subscription status, and discounts.  
3. Measure the impact of promotions and payment preferences.  
4. Identify high-value customers and product combinations.  

## Dataset Overview  
The dataset includes detailed customer shopping information:

- Demographics: age, gender  
- Product details: item, category, color, size  
- Behavioral data: purchase amount, previous purchases, frequency  
- Marketing factors: discounts, promo usage, subscription status  
- Operational factors: season, shipping type, payment method  

This dataset enables a comprehensive understanding of customer behavior and revenue drivers.

## Notebook Structure
1. Database connection setup  
2. KPI summary  
3. Core SQL analysis  






In [1]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import pandas as pd
from IPython.display import display

user = "khirod"
pwd = "Password123!"   
host = "localhost"
port = 3306
db = "customer_behavior_analysis"

uri = f"mysql+pymysql://{user}:{quote_plus(pwd)}@{host}:{port}/{db}"

engine = create_engine(uri)

print("Connected successfully!")


Connected successfully!


In [24]:
kpi_query = """
SELECT
    ROUND(SUM(purchase_amount),2) AS total_revenue,
    ROUND(AVG(purchase_amount),2) AS avg_order_value,
    ROUND(
      100.0 * SUM(CASE WHEN previous_purchases > 1 THEN 1 ELSE 0 END) / COUNT(*),
      2
    ) AS repeat_customer_rate,
    ROUND(
      100.0 * SUM(CASE WHEN subscription_status = 'Yes' THEN purchase_amount ELSE 0 END) /
      SUM(purchase_amount),
      2
    ) AS subscriber_revenue_pct
FROM customer;
"""

df_kpi = pd.read_sql(kpi_query, engine)
display(df_kpi)


,total_revenue,avg_order_value,repeat_customer_rate,subscriber_revenue_pct
0,233081.0,59.76,97.87,26.88


**Insight:**  
The business has generated a total revenue of 233,081 with an average order value of around 60. A very high repeat customer rate indicates strong customer retention, while subscribers contribute a meaningful share of overall revenue.


In [4]:
q1 = """
SELECT gender, SUM(purchase_amount) AS revenue
FROM customer
GROUP BY gender;
"""
df_q1 = pd.read_sql(q1, engine)
display(df_q1)


,gender,revenue
0,Male,157890.0
1,Female,75191.0


**Insight:**  
Male customers contribute a significantly higher share of total revenue compared to female customers. This suggests that current products or engagement strategies are more effective for male audiences.


In [41]:
q2_count = """
SELECT
    COUNT(DISTINCT CASE 
        WHEN discount_applied = 'Yes'
         AND purchase_amount >= (
             SELECT AVG(purchase_amount)
             FROM customer
             WHERE discount_applied = 'Yes'
         )
        THEN customer_id
    END) AS high_value_discount_customers,
    ROUND(
        100.0 * COUNT(DISTINCT CASE 
            WHEN discount_applied = 'Yes'
             AND purchase_amount >= (
                 SELECT AVG(purchase_amount)
                 FROM customer
                 WHERE discount_applied = 'Yes'
             )
            THEN customer_id
        END) / COUNT(DISTINCT customer_id),
        2
    ) AS pct_of_customers
FROM customer;

"""

df_q2_count = pd.read_sql(q2_count, engine)
display(df_q2_count)


,high_value_discount_customers,pct_of_customers
0,839,21.51


**Insight:**  
Only 21% of customers are high-value discount users, indicating that most customers do not combine discounts with above-average spending. This suggests that discounts are not the primary driver of high-value purchases and may be used selectively rather than broadly.


In [8]:
q3 = """
SELECT item_purchased,
       ROUND(AVG(review_rating), 2) AS avg_product_rating
FROM customer
GROUP BY item_purchased
ORDER BY AVG(review_rating) DESC
LIMIT 5;
"""
df_q3 = pd.read_sql(q3, engine)
display(df_q3)


,item_purchased,avg_product_rating
0,Gloves,3.86
1,Sandals,3.84
2,Boots,3.82
3,Hat,3.80
4,Skirt,3.78


**Insight:**  
Gloves, Sandals, and Boots have the highest average customer ratings, indicating strong customer satisfaction with these products. These items represent high-quality offerings that could be prioritized for promotion or inventory planning.


In [10]:
q4 = """
SELECT shipping_type,
       ROUND(AVG(purchase_amount), 2) AS avg_purchase
FROM customer
WHERE shipping_type IN ('Standard','Express')
GROUP BY shipping_type;
"""
df_q4 = pd.read_sql(q4, engine)
display(df_q4)


,shipping_type,avg_purchase
0,Express,60.48
1,Standard,58.46


**Insight:**  
Customers using Express shipping have a slightly higher average purchase amount compared to those using Standard shipping. This suggests that customers opting for faster delivery may be more willing to spend marginally more per order.


In [12]:
q5 = """
SELECT subscription_status,
       COUNT(customer_id) AS total_customers,
       ROUND(AVG(purchase_amount),2) AS avg_spend,
       ROUND(SUM(purchase_amount),2) AS total_revenue
FROM customer
GROUP BY subscription_status
ORDER BY total_revenue DESC, avg_spend DESC;
"""
df_q5 = pd.read_sql(q5, engine)
display(df_q5)


,subscription_status,total_customers,avg_spend,total_revenue
0,No,2847,59.87,170436.0
1,Yes,1053,59.49,62645.0


**Insight:**  
Non-subscribers generate higher total revenue primarily because they represent a much larger customer base, while average spending per order is nearly the same for both subscribers and non-subscribers. This suggests subscription status currently impacts customer volume more than per-order spend.


In [14]:
q6 = """
SELECT item_purchased,
       ROUND(
         100.0 * SUM(CASE WHEN discount_applied = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),
         2
       ) AS discount_rate
FROM customer
GROUP BY item_purchased
ORDER BY discount_rate DESC
LIMIT 5;
"""
df_q6 = pd.read_sql(q6, engine)
display(df_q6)


,item_purchased,discount_rate
0,Hat,50.00
1,Sneakers,49.66
2,Coat,49.07
3,Sweater,48.17
4,Pants,47.37


**Insight:**  
Hats, Sneakers, and Coats have the highest discount usage rates, indicating these products rely more heavily on promotions to drive sales. This suggests pricing or demand for these items may be more sensitive compared to other products.


In [16]:
q7 = """
WITH customer_type AS (
    SELECT customer_id,
           CASE
               WHEN previous_purchases = 1 THEN 'New'
               WHEN previous_purchases BETWEEN 2 AND 10 THEN 'Returning'
               ELSE 'Loyal'
           END AS customer_segment
    FROM customer
)
SELECT customer_segment, COUNT(*) AS number_of_customers
FROM customer_type
GROUP BY customer_segment;
"""
df_q7 = pd.read_sql(q7, engine)
display(df_q7)


,customer_segment,number_of_customers
0,Loyal,3116
1,Returning,701
2,New,83


**Insight:**  
The customer base is dominated by loyal customers, while new customers form a very small proportion. This indicates strong retention but also suggests an opportunity to improve customer acquisition efforts.


In [18]:
q8 = """
WITH item_counts AS (
    SELECT category,
           item_purchased,
           COUNT(customer_id) AS total_orders,
           ROW_NUMBER() OVER (
               PARTITION BY category
               ORDER BY COUNT(customer_id) DESC
           ) AS item_rank
    FROM customer
    GROUP BY category, item_purchased
)
SELECT item_rank, category, item_purchased, total_orders
FROM item_counts
WHERE item_rank <= 3;
"""
df_q8 = pd.read_sql(q8, engine)
display(df_q8)


,item_rank,category,item_purchased,total_orders
0,1,Accessories,Jewelry,171
1,2,Accessories,Sunglasses,161
2,3,Accessories,Belt,161
3,1,Clothing,Blouse,171
4,2,Clothing,Pants,171
5,3,Clothing,Shirt,169
6,1,Footwear,Sandals,160
7,2,Footwear,Shoes,150
8,3,Footwear,Sneakers,145
9,1,Outerwear,Jacket,163


**Insight:**  
Each product category shows a small set of consistently high-performing items that account for most purchases. This indicates clear customer preferences within categories, which can be leveraged for focused inventory planning and targeted promotions.


In [20]:
q9 = """
SELECT subscription_status,
       COUNT(customer_id) AS repeat_buyers
FROM customer
WHERE previous_purchases > 5
GROUP BY subscription_status;
"""
df_q9 = pd.read_sql(q9, engine)
display(df_q9)


,subscription_status,repeat_buyers
0,Yes,958
1,No,2518


**Insight:**  
Most repeat buyers are non-subscribers, indicating that frequent purchasing does not automatically translate into subscription adoption. This highlights an opportunity to better target loyal customers with subscription-related incentives.


In [22]:
q10 = """
SELECT age_group,
       SUM(purchase_amount) AS total_revenue
FROM customer
GROUP BY age_group
ORDER BY total_revenue DESC;
"""
df_q10 = pd.read_sql(q10, engine)
display(df_q10)


,age_group,total_revenue
0,Young Adult,62143.0
1,Middle-aged,59197.0
2,Adult,55978.0
3,Senior,55763.0


**Insight:**  
Young Adult customers contribute the highest total revenue, closely followed by Middle-aged and Adult groups. Revenue is fairly evenly distributed across age segments, suggesting broad appeal across different age groups rather than dependence on a single segment.
